In [28]:
import os
from dotenv import load_dotenv

from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain import hub

In [2]:
load_dotenv()

True

In [9]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=wiki_wrapper)


In [4]:
loader = WebBaseLoader("https://docs.langchain.com/langsmith/home")
docs = loader.load()
doc_splits = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 200).split_documents(docs)
vector_db = FAISS.from_documents(doc_splits, OpenAIEmbeddings())
retriever = vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FB9FD2E9D0>, search_kwargs={})

In [5]:
retrieval_tool = create_retriever_tool(retriever, "langsmith_search",
                                       "Search for information about LangSmith, for any langsmith search you must use this tool")

In [8]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [10]:
tools = [wiki, arxiv, retrieval_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\shaur\\Desktop\\Learnings\\Langchain\\lang_env\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith, for any langsmith search you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001FB9E5F2F20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langcha

In [29]:
llm = ChatOpenAI(model="gpt-5-nano", temperature=0)
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [30]:
### Agents
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)

## Agent Executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [31]:
agent_executor.invoke({
    "input": "Tell me about langsmith"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


LangSmith is framework agnostic — you can use it with or without LangChain’s open source frameworks langchain and langgraph.

Get started with LangSmith - Docs by LangChainOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KGitHubForumForumSearch...NavigationGet started with LangSmithGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGitHubForumGet started with LangSmithCopy pageCopy pageLangSmith is a plat

{'input': 'Tell me about langsmith',
 'output': 'LangSmith is a platform (by LangChain) for building production-grade LLM applications. It’s framework-agnostic, so you can use it with or without LangChain’s open-source frameworks like LangChain or LangGraph. Its main goal is to help you monitor, evaluate, and iterate on your AI applications with confidence.\n\nKey capabilities\n- Tracing: Get visibility into each step of a request to debug faster.\n- Evaluation: Measure quality over time to ship more reliable AI apps.\n- Prompt testing: Iterate on prompts with version control and collaboration features.\n- Versioned prompts: Automatic versioning so you can track changes and compare prompts.\n- Collaboration and admin: Set up workspaces, invite teammates, and manage access.\n- Self-hosting: Options to self-host for more control.\n- SDKs and integrations: API and SDKs for Python and JS/TS; works with or without LangChain/LangGraph.\n\nWhat to do next\n- Start with the LangSmith docs to s

In [32]:
agent_executor.invoke({
    "input": "Tell me about machine learning"
})



> Entering new AgentExecutor chain...
Here’s a concise overview of machine learning (ML) and how it fits together.

What is machine learning?
- ML is the practice of building systems that learn from data to make predictions or decisions, rather than following explicit, hand-written rules.

Core types of ML
- Supervised learning: learn from labeled data to predict a target. Tasks include:
  - Classification (e.g., spam vs. not spam)
  - Regression (e.g., predicting house prices)
- Unsupervised learning: learn from unlabeled data to find structure or patterns.
  - Clustering (e.g., customer segmentation)
  - Dimensionality reduction (e.g., reducing features for visualization)
- Semi-supervised learning: uses mostly unlabeled data with a small amount of labeled data.
- Reinforcement learning: an agent learns by interacting with an environment and maximizing cumulative rewards (common in robotics, games).

How ML works at a high level
1. Define the problem and objective (what you want to

{'input': 'Tell me about machine learning',
 'output': 'Here’s a concise overview of machine learning (ML) and how it fits together.\n\nWhat is machine learning?\n- ML is the practice of building systems that learn from data to make predictions or decisions, rather than following explicit, hand-written rules.\n\nCore types of ML\n- Supervised learning: learn from labeled data to predict a target. Tasks include:\n  - Classification (e.g., spam vs. not spam)\n  - Regression (e.g., predicting house prices)\n- Unsupervised learning: learn from unlabeled data to find structure or patterns.\n  - Clustering (e.g., customer segmentation)\n  - Dimensionality reduction (e.g., reducing features for visualization)\n- Semi-supervised learning: uses mostly unlabeled data with a small amount of labeled data.\n- Reinforcement learning: an agent learns by interacting with an environment and maximizing cumulative rewards (common in robotics, games).\n\nHow ML works at a high level\n1. Define the problem